# 🎨🌞 Albedo Experiment

### Originally created by Dr. A. Denvil-Sommer (NCAS, Univerity of Reading).

## 🌊 Ocean Albedo and Climate Sensitivity

The surface albedo of the global ocean is a critical factor in Earth’s climate system. Even small changes in ocean albedo can significantly alter the global energy budget, which in turn impacts atmospheric circulation, ocean currents, and the hydrological cycle.

Typical ocean albedo values range from 0.06 to 0.10, with lower values near the equator and slightly higher values near the poles due to changes in sun angle and surface roughness.

## 🎯 Objective

Investigate how increasing ocean **albedo (reflectivity)** affects surface temperature and the atmosphere when coupled with land and ocean models in SpeedyWeather.jl.

## 🌊🧪 Ocean Albedo Experiment Setup
We will explore how changes in ocean albedo affect the atmosphere by comparing two simulations:

**🔹 Control Run (Realistic Albedo)**

Set a typical global ocean albedo with a slight increase toward the poles:
`set!(albedo.ocean, (λ, φ) -> 0.06 + 0.04 * abs(φ)/90)`.

**🔸 Perturbed Run (High Albedo)**

Now increase the ocean albedo uniformly to examine its impact:
`set!(albedo.ocean, (λ, φ) -> 0.16 + 0.04 * abs(φ)/90)`. 

⚠️ Important: Save each simulation to a separate folder or give output files unique names, so you can easily compare results later without overwriting.

## 🔬 🧠 Guiding Questions

- How did the surface temperature respond to increased ocean albedo?

- Do the equator-to-pole temperature gradients change?

- Is there a noticeable impact on precipitation patterns?

- How do atmospheric circulation fields (e.g., zonal wind) respond?

- Can you relate these effects to Earth’s climate sensitivity or feedbacks?

## 💾 Variables to Invistigate 

To investigate the effects of ocean albedo changes, you should save the following variables from your simulation output:

- **🌊 Sea Surface Temperature (SST)**
`simulation.prognostic_variables.ocean.sea_surface_temperature`

- **💧 Specific Humidity**
`simulation.diagnostic_variables.grid.humid_grid`

- **🌬️ Surface Winds (Zonal and Meridional)**
`simulation.diagnostic_variables.grid.u_grid`
`simulation.diagnostic_variables.grid.v_grid`

- **🌧️ Precipitation (Large-scale)**
`simulation.diagnostic_variables.physics.precip_large_scale`

- **📉 Pressure Tendency (for Hadley Cell analysis)**
`simulation.diagnostic_variables.tendencies.pres_tend`

👉 You may also consider saving additional variables depending on the focus of your analysis.

In [ ]:
using SpeedyWeather
using Dates
using CairoMakie

In [ ]:
# 1. Define the grid and components
spectral_grid = SpectralGrid(trunc=21, nlayers=8)
output = NetCDFOutput(spectral_grid, PrimitiveWetModel, id="Climate_Ocean_Albedo", output_dt=Minute(180))
add!(output, SpeedyWeather.TemperatureOutput(), SpeedyWeather.LargeScalePrecipitationOutput(), SpeedyWeather.ConvectivePrecipitationOutput())  # output also temperature and precip

In [ ]:
# 2. Create land and ocean models
vertical_advection = WENOVerticalAdvection(spectral_grid)
large_scale_condensation = ImplicitCondensation(spectral_grid)
convection = SimplifiedBettsMiller(spectral_grid)
temperature=LandBucketTemperature(spectral_grid)
soil_moisture=LandBucketMoisture(spectral_grid)
land = LandModel(spectral_grid; temperature, soil_moisture)
ocean = SlabOcean(spectral_grid, mixed_layer_depth=50)

In [ ]:
# 3. Set up albedo component
#albedo = Albedo(GlobalConstantAlbedo(spectral_grid, albedo=0.06), AlbedoClimatology(spectral_grid))
# create model, initialize, run
# 3. Set up albedo component
# Create a ManualAlbedo object
manual = ManualAlbedo(spectral_grid)

# Create the composite albedo object
albedo = Albedo(ocean=manual, land=AlbedoClimatology(spectral_grid))

# manual albedo for the ocean
set!(albedo.ocean, (λ, φ) -> 0.16 + 0.04 * abs(φ)/90)

In [ ]:
# create model, initialize, run
time_stepping = Leapfrog(spectral_grid, Δt_at_T31=Minute(40)) #CPL this is the standar time step

### Entirely new run
model = PrimitiveWetModel(spectral_grid; vertical_advection, time_stepping, large_scale_condensation, convection, land, ocean, albedo, output=output)

add!(model, SpeedyWeather.AllOutputVariables()...)
simulation = initialize!(model)
# now offset via set! with add=true
set!(albedo.land, (λ, φ) -> 0., add=true)
set!(albedo.ocean, (λ, φ) -> 0.06 + 0.04 * abs(φ)/90, add=true)

In [ ]:
# visualise intial land albedo
alb = simulation.model.albedo.land.albedo
heatmap(alb, title="albedo")

In [ ]:
# visualise intial ocean albedo
alb = simulation.model.albedo.ocean.albedo
heatmap(alb, title="albedo")

In [ ]:
# run the simualtion
run!(simulation, period=Day(30*365), output=true)

In [ ]:
alb = simulation.model.albedo.land.albedo
heatmap(alb, title="albedo")

In [ ]:
alb = simulation.model.albedo.ocean.albedo
heatmap(alb, title="albedo")

In [ ]:
alb = simulation.diagnostic_variables.physics.ocean.albedo
heatmap(alb, title="albedo")

In [ ]:
alb_land = simulation.diagnostic_variables.physics.land.albedo
heatmap(alb_land, title="albedo")

In [ ]:
sst = simulation.prognostic_variables.ocean.sea_surface_temperature
heatmap(sst, title="Surface Temperature [K]")

In [ ]:
T = simulation.diagnostic_variables.grid.temp_grid[:, 8]
heatmap(T, title="Surface Temperature [K]")

In [ ]:
P = simulation.diagnostic_variables.physics.precip_large_scale
heatmap(P, title="Precipitation [mm/day]")

In [ ]:
U = simulation.diagnostic_variables.grid.u_grid[:, 8]
heatmap(U, title="Zonal Wind at Surface [m/s]")